<a href="https://colab.research.google.com/github/jetsonmom/6.23_automobility_lesson/blob/main/6_23_%EA%B0%95%EC%9D%98_step2_1%EB%8B%A8%EA%B3%84_%EA%B0%84%EB%8B%A8%ED%95%9C_%EC%B0%A8%EC%84%A0_%EB%94%B0%EB%9D%BC%EA%B0%80%EA%B8%B0_%EC%95%A0%EB%8B%88%EB%A9%94%EC%9D%B4%EC%85%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1단계: 간단한 차선 따라가기 애니메이션

In [ ]:
# 첫 번째 셀 - 라이브러리 설치
!pip install matplotlib
!pip install numpy
!pip install IPython

# 애니메이션이 안 되면 추가 설치
!apt-get update
!apt-get install -y ffmpeg

In [ ]:
# 🚗 1단계: 기본 차선 유지 시뮬레이션
# Google Colab에서 실행 가능한 애니메이션

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import time

# 기본 설정
plt.style.use('dark_background')  # 멋진 다크 테마
plt.rcParams['figure.facecolor'] = 'black'

class LaneKeepingSimulator:
    def __init__(self):
        # 도로 설정
        self.road_width = 100
        self.road_length = 200
        self.lane_width = 15

        # 차량 초기 위치
        self.car_x = 50  # 도로 중앙
        self.car_y = 20  # 시작 지점
        self.car_speed = 2

        # 차선 위치 (좌측, 중앙, 우측)
        self.left_lane = 35
        self.center_lane = 50
        self.right_lane = 65

        # 제어 변수
        self.steering_angle = 0
        self.target_lane = self.center_lane

        # 기록용
        self.positions = []
        self.steering_history = []

    def create_road(self):
        """도로와 차선 생성"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

        # 메인 시뮬레이션 화면
        ax1.set_xlim(0, self.road_width)
        ax1.set_ylim(0, 150)
        ax1.set_facecolor('black')
        ax1.set_title('🚗 Lane Keeping Simulation',
                     fontsize=16, color='white', fontweight='bold')
        ax1.set_xlabel('Road Width (m)', color='white')
        ax1.set_ylabel('Distance (m)', color='white')

        # 도로 그리기 (회색 아스팔트)
        road = plt.Rectangle((20, 0), 60, 150,
                           facecolor='#2C2C2C', edgecolor='white', linewidth=2)
        ax1.add_patch(road)

        # 차선 그리기 (하얀 점선)
        lane_y = np.arange(0, 150, 5)
        ax1.plot([self.left_lane]*len(lane_y), lane_y,
                'w--', linewidth=2, alpha=0.8, label='Left Lane')
        ax1.plot([self.center_lane]*len(lane_y), lane_y,
                'w--', linewidth=2, alpha=0.8, label='Center Lane')
        ax1.plot([self.right_lane]*len(lane_y), lane_y,
                'w--', linewidth=2, alpha=0.8, label='Right Lane')

        # 차량 (빨간 점으로 시작)
        self.car_dot, = ax1.plot(self.car_x, self.car_y, 'ro',
                                markersize=15, label='Vehicle')

        # 차량 경로 (초록 선)
        self.path_line, = ax1.plot([], [], 'g-', linewidth=3, alpha=0.7,
                                  label='Vehicle Path')

        # 범례
        ax1.legend(loc='upper right')
        ax1.grid(True, alpha=0.3)

        # 제어 정보 화면
        ax2.set_facecolor('black')
        ax2.set_title('📊 Control Information',
                     fontsize=16, color='white', fontweight='bold')
        ax2.set_xlim(0, 100)
        ax2.set_ylim(0, 100)

        # 실시간 정보 텍스트
        self.info_text = ax2.text(10, 80, '', fontsize=12, color='cyan',
                                 family='monospace')

        # 조향각 그래프
        self.steering_line, = ax2.plot([], [], 'y-', linewidth=2)
        ax2.set_xlabel('Time Steps', color='white')
        ax2.set_ylabel('Steering Angle', color='white')

        return fig, ax1, ax2

    def simple_controller(self):
        """간단한 PID 제어기"""
        # 목표 차선과의 거리 오차
        error = self.target_lane - self.car_x

        # 비례 제어 (P control)
        kp = 0.1  # 비례 상수
        self.steering_angle = kp * error

        # 조향각 제한 (-5도 ~ +5도)
        self.steering_angle = np.clip(self.steering_angle, -5, 5)

        return self.steering_angle

    def update_vehicle(self):
        """차량 위치 업데이트"""
        # 조향각에 따른 횡방향 이동
        lateral_movement = self.steering_angle * 0.3

        # 차량 위치 업데이트
        self.car_x += lateral_movement
        self.car_y += self.car_speed

        # 도로 경계 체크
        self.car_x = np.clip(self.car_x, 22, 78)

        # 경로 기록
        self.positions.append([self.car_x, self.car_y])
        self.steering_history.append(self.steering_angle)

    def animate(self, frame):
        """애니메이션 업데이트 함수"""
        # 제어기 실행
        steering = self.simple_controller()

        # 차량 업데이트
        self.update_vehicle()

        # 차량 위치 업데이트
        self.car_dot.set_data([self.car_x], [self.car_y])

        # 경로 표시
        if len(self.positions) > 1:
            path_x = [pos[0] for pos in self.positions[-20:]]  # 최근 20개점만
            path_y = [pos[1] for pos in self.positions[-20:]]
            self.path_line.set_data(path_x, path_y)

        # 제어 정보 업데이트
        info = f"""🚗 Vehicle Status:

Position X: {self.car_x:.1f} m
Position Y: {self.car_y:.1f} m
Speed: {self.car_speed:.1f} m/s
Steering: {self.steering_angle:.2f}°

Target Lane: {self.target_lane:.1f} m
Error: {self.target_lane - self.car_x:.2f} m

Frame: {frame}
        """
        self.info_text.set_text(info)

        # 조향각 히스토리 그래프
        if len(self.steering_history) > 1:
            self.steering_line.set_data(range(len(self.steering_history)),
                                      self.steering_history)

        # 화면 스크롤 (차량이 위로 올라가면)
        if self.car_y > 100:
            self.car_y = 20
            self.positions = []  # 경로 리셋

        return self.car_dot, self.path_line, self.info_text, self.steering_line

# 🎮 시뮬레이션 실행
print("🚗 Lane Keeping Simulation Starting...")
print("차량이 중앙 차선을 유지하며 주행합니다!")

# 시뮬레이터 생성
simulator = LaneKeepingSimulator()
fig, ax1, ax2 = simulator.create_road()

# 애니메이션 생성 (30프레임, 무한 반복)
anim = animation.FuncAnimation(
    fig,
    simulator.animate,
    interval=100,  # 100ms마다 업데이트
    frames=200,    # 200프레임
    blit=False,
    repeat=True
)

# Colab에서 애니메이션 표시
plt.tight_layout()
plt.show()

# 애니메이션을 HTML로 변환 (Colab에서 재생 가능)
print("\n🎬 애니메이션 생성 중... (잠시만 기다려주세요)")
html_anim = HTML(anim.to_jshtml())

print("✅ 시뮬레이션 완료!")
print("📊 차량이 PID 제어로 차선을 유지하며 주행했습니다!")

# 결과 통계
print(f"\n📈 시뮬레이션 결과:")
print(f"최대 조향각: {max(simulator.steering_history):.2f}°")
print(f"평균 조향각: {np.mean(simulator.steering_history):.2f}°")
print(f"주행 거리: {len(simulator.positions) * simulator.car_speed:.1f}m")

# HTML 애니메이션 반환
html_anim